In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import os
from PIL import Image, UnidentifiedImageError
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.io import read_image
from pdb import set_trace
from torch.optim.lr_scheduler import StepLR

In [2]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0], std=[1])  # Normalize to [-1, 1]
])

In [3]:
root_dir = "../input/microsoft-catsvsdogs-dataset/PetImages"

In [4]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, root_dir, transform=None, start=0, finish=1000):
        self.root_dir = root_dir
        self.transform = transform
        
        self.dog_files = os.listdir(os.path.join(root_dir, "Dog"))[start:finish]
        self.cat_files = os.listdir(os.path.join(root_dir, "Cat"))[start:finish]
        
        self.length = min(len(self.dog_files), len(self.cat_files))
    
    def __len__(self):
        return self.length * 2
    

    def __getitem__(self, idx):
        try:
            if idx % 2 == 0:
                folder = "Dog"
                image_files = self.dog_files
                label = 1
            else:
                folder = "Cat"
                image_files = self.cat_files
                label = 0  # Cat label
            
            adjusted_idx = idx // 2
            img_path = os.path.join(self.root_dir, folder, image_files[adjusted_idx])
            
            image = Image.open(img_path).convert("RGB")
            
            if self.transform:
                image = self.transform(image)
            
            return image, label
            
        except (UnidentifiedImageError, OSError) as e:
            print(f"Skipping corrupted image: {img_path}")
            return self.__getitem__((idx + 2) % len(self))


In [5]:
# Create datasets
train_dataset = CatsAndDogsDataset(root_dir=root_dir, transform=transform, finish=8000)
val_dataset = CatsAndDogsDataset(root_dir=root_dir, transform=transform, start=8000, finish=9000)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [6]:
import torch.nn as nn

class FullyConnectedNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_prob=0.5):
        super().__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_sizes[0]),
            nn.Dropout(dropout_prob), 
            
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_sizes[1]),
            nn.Dropout(dropout_prob), 
            
            nn.Linear(hidden_sizes[1], output_size),
            nn.Sigmoid() 
        )
    
    def forward(self, x):
        return self.fc_layers(x)


In [7]:
input_size = 3 * 64 * 64
hidden_sizes = [512, 128]  # hidden layer sizes
output_size = 1  

In [8]:
model = FullyConnectedNetwork(input_size, hidden_sizes, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

FullyConnectedNetwork(
  (fc_layers): Sequential(
    (0): Linear(in_features=12288, out_features=512, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=128, out_features=1, bias=True)
    (9): Sigmoid()
  )
)

In [9]:
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Reduce LR by 0.1 every 10 epochs

In [10]:
%%time

epochs = 1
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # images = images.to(device)
        # labels = labels.float().unsqueeze(1).to(device)
        images = images.view(images.size(0), -1).to(device)  # Flatten
        labels = labels.float().unsqueeze(1).to(device)  # Match output shape

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    scheduler.step()
    # if epoch % 10 == 0:
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Skipping corrupted image: ../input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg
Epoch 1/1, Loss: 0.6941
CPU times: user 35.4 s, sys: 1.93 s, total: 37.3 s
Wall time: 1min 4s


In [11]:
%%time
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        try:
            images = images.view(images.size(0), -1).to(device)
            labels = labels.to(device)
            outputs = model(images)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
            
        except Exception as e:
            print('exception raised', e)
            pass

    print(f"Validation Accuracy: {100 * correct / (total + 0.000001):.2f}%")

Skipping corrupted image: ../input/microsoft-catsvsdogs-dataset/PetImages/Dog/Thumbs.db
Skipping corrupted image: ../input/microsoft-catsvsdogs-dataset/PetImages/Cat/Thumbs.db
Validation Accuracy: 57.80%
CPU times: user 4.58 s, sys: 242 ms, total: 4.82 s
Wall time: 6.49 s


In [12]:
torch.save(model, 'test.pth')